In [ ]:
import matplotlib
from datascience import *
%matplotlib inline
import matplotlib.pyplot as plots
import numpy as np
plots.style.use('fivethirtyeight')

In [ ]:
# Some functions for plotting. You don't have to understand how any
# of the functions in this cell work, since they use things we 
# haven't learned about.

def resize_window(lim=3.5):
    plots.xlim(-lim, lim)
    plots.ylim(-lim, lim)
    
def draw_line(slope=0, intercept=0, x=make_array(-4, 4), color='#1e90ff'):
    y = x*slope + intercept
    plots.plot(x, y, color=color, lw=3)
    
def draw_vertical_line(x_position, color='black'):
    x = make_array(x_position, x_position)
    y = make_array(-4, 4)
    plots.plot(x, y, color=color, lw=3)
    
def make_correlated_data(r):
    x = np.random.normal(0, 1, 1000)
    z = np.random.normal(0, 1, 1000)
    y = r*x + (np.sqrt(1-r**2))*z
    return x, y

def r_scatter(r):
    """Generate a scatter plot with a correlation approximately r"""
    plots.figure(figsize=(5,5))
    x, y = make_correlated_data(r)
    plots.scatter(x, y, color='darkblue', s=20)
    plots.xlim(-4, 4)
    plots.ylim(-4, 4)
    
def r_table(r):
    """
    Generate a table of 1000 data points with a correlation approximately r
    """
    np.random.seed(8)
    x, y = make_correlated_data(r)
    return Table().with_columns('x', x, 'y', y)

# Lecture 30: Linear Regression

## A. Review of Correlation

Define the standard_units function.

In [ ]:
def standard_units(x):
    "Converts any array of numbers to standard units."
    return ...

x = make_array(1, 2, 3, 3, 3, 4, 5)
standard_units(x)

Define the correlation function.

In [ ]:
# Correlation is defined to be the average product of 
# standard values for x and y
def correlation(t, x, y):
    """
    t is a table; x and y are column labels for t (numerical data);
    returns the correlation between x and y
    """
    x_in_standard_units = ...  # use previous function on the column for x
    y_in_standard_units = ...  # use previous function on the column for y
    return ...

t = Table().with_columns(
    'x', make_array(1, 2, 3, 4), 
    'y', make_array(2, 3.5, 4, 7)
)
correlation(t, 'x', 'y')

In [ ]:
# Recall the data about hybrid cars
hybrid = Table.read_table('hybrid.csv')
hybrid.show(3)

In [ ]:
# Visualize the association between acceleration and msrp
hybrid.scatter('acceleration', 'msrp')

In [ ]:
# The association is moderate, positive, and linear.
# What is the correlation?
correlation(hybrid, 'acceleration', 'msrp')

### Switching Axes
Recall that for the correlation, it does not matter which variable is 'x' and which is 'y'. The correlation is not affected when we swap the axes.

In [ ]:
# swapping axes reflects the dots through the line y = x
hybrid.scatter('msrp', 'acceleration')

In [ ]:
# The correlation doesn't depend on which variable is 'x' and which is 'y'
correlation(hybrid, 'msrp', 'acceleration')

### Nonlinearity
Correlation does NOT measure the association between variables if the association is strongly non-linear. We saw this last time:

In [ ]:
# An example where y = x^2 : a perfect parabola
# If you tell me x, I can tell you y exactly -- a very strong nonlinear association
new_x = np.arange(-4, 4.1, 0.5)
nonlinear = Table().with_columns(
        'x', new_x,
        'y', new_x**2
    )
nonlinear.scatter('x', 'y', s=30, color='r')

In [ ]:
# For this nonlinear association, what's the correlation?
correlation(nonlinear, 'x', 'y')

### Outliers
Correlation is very sensitive to outliers. Especially if there are not many data points, one extreme outlier can bring the correlation to zero.

In [ ]:
# A perfect positive correlation: y = x
line = Table().with_columns(
        'x', make_array(1, 2, 3, 4),
        'y', make_array(1, 2, 3, 4)
    )
line.scatter('x', 'y', s=30, color='r')

In [ ]:
correlation(line, 'x', 'y')

In [ ]:
# A perfect positive correlation, plus one extreme outlier at (5,0)
outlier = Table().with_columns(
        'x', make_array(1, 2, 3, 4, 5),
        'y', make_array(1, 2, 3, 4, 0)
    )
outlier.scatter('x', 'y', s=30, color='r')

In [ ]:
# Now the correlation is zero.
# This is why we have the motto: Visualize FIRST, then calculate numerical summaries SECOND
# We need a visual context to understand the correlation
correlation(outlier, 'x', 'y')

If this happens in real life, don't just throw out the outlier and move on with your life!

  - If the outlier is a correct data point, it tells you something interesting about the world. Investigate it. Outliers are often the most interesting points in a data set!
  - If the outlier is due to a typo, fix it.

In [ ]:
# Back to slides...






## B. Ecological Correlations
Here's an interesting real-world example. We have data on the Critical Reading and Math SAT scores in 2014. There is one point for each of the 50 states and one for Washington, D.C. 

The column `Participation Rate` indicates the percent of high school seniors who took the test. The next three columns show the average score in the state on each portion of the test, and the final column is the average of the total scores on the test.

In [ ]:
sat2014 = Table.read_table('sat2014.csv').sort('State')
sat2014

In [ ]:
# On a state-by-state basis, is there an association between mean critical 
# reading score and mean math score on the SAT?
sat2014.scatter('Critical Reading', 'Math')

In [ ]:
# Guess the correlation from the plot, then calculate it
correlation(sat2014, 'Critical Reading', 'Math')

What does this tell us about an individual student? Is there such a strong relationship between critical reading and math scores on the SAT? Probably not. 

Discuss: Why is there such a wide range of scores across the states?

Some states have a high partipation rate, some have a moderate participation rate, and some have a low participation rate. When there's a low participation rate, you can bet that most of the students taking the SAT are headed for college.

In [ ]:
def rate_code(x):
    '''Return a string which classifies the participation rate x 
    as low, medium, or high'''
    if x <= 25:
        return '3: low'
    elif x <= 75:
        return '2: medium'
    else:
        return '1: high'

In [ ]:
# Use the tbl.apply() method to compute an array of rate codes for 
# participation in the SAT for various states.
rate_codes = sat2014.apply(rate_code, 'Participation Rate')

In [ ]:
# Add a Rate Code column to the `sat2014` table
sat2014 = sat2014.with_column('Rate Code', rate_codes)

# Sort into descending order by participation rate
sat2014 = sat2014.sort('Participation Rate', descending = True)
sat2014

In [ ]:
# Discuss what you learn about the world from this visualization
sat2014.scatter('Critical Reading', 'Math', group='Rate Code')

In [ ]:
# Scope out the states with a low participation rate
sat2014.where('Rate Code', '3: low').show()

In [ ]:
# Group the states by Rate Code and compare their mean results
sat2014.group('Rate Code', np.mean)

Takeaway: If you want your state to look good on the SAT, only have the strongest students take the test!

Remember: An "Ecological Correlation" is based on aggregated data, and does NOT help us understand the association between the variables for **individuals**. Correlation must be interpreted carefully.

## C. Prediction Lines

### r = 0.99

In [ ]:
# Use r_table to build a table with columns x and y such that r = 0.99
example = r_table(0.99)
example.show(3)

In [ ]:
# Scatter plot for our example with r = 0.99
# It's apparent that the slope of the fit line is 1; notice the grid points
# (-3, -3) and (3, 3) are on the fit line.
example.scatter('x', 'y')
resize_window()

In [ ]:
# Use our "nearest neighbors" prediction method, as for family heights
# Define a function which takes an x value and returns a predicted y value

def nn_prediction_example(x_val):
    """ Predicts y-value for x based on the example table """
    neighbors = example.where('x', are.between(x_val - .25, x_val + .25))
    return np.mean(neighbors.column('y'))   

In [ ]:
nn_prediction_example(-2.25)

In [ ]:
# Add a 'Predicted y' column to our example table, r = 0.99
example = example.with_columns(
    'Predicted y', 
    example.apply(nn_prediction_example, 'x'))

In [ ]:
# We didn't impose the linear shape on the predictions
# The linear shape is a natural result of the shape of the histogram
example.scatter('x')
resize_window()

In [ ]:
# The fit line with slope 1 through (0, 0) is a good match to the predictions
example.scatter('x')
draw_line(slope=1)
resize_window()

### r = 0
Ok, that's great. But what if the correlation is zero?

In [ ]:
# Here's a new example, r = 0.0
example = r_table(0)
example.scatter('x', 'y')
resize_window()

In [ ]:
# Add the nearest-neighbor predictions
example = example.with_columns(
    'Predicted y', 
    example.apply(nn_prediction_example, 'x'))
example.show(3)

In [ ]:
# Visualize the example data and the predictions
example.scatter('x')
resize_window()

On the ends of the prediction "curve", things get a bit sketchy due to averaging just a few extreme points to make predictions. But overall, the prediction curve suggests a horizontal fit line (slope = 0) through (0, 0).

In [ ]:
# Visualize the example data and the predictions, plus the line with slope 0
example.scatter('x')
draw_line(slope=0)
resize_window()

### r = 0.5
A more typical correlation in real data is r = 0.5.  What happens here?

In [ ]:
# An example table with r = 0.5
# We see an oval scatter plot with a lot of scatter and a positive trend
example = r_table(0.5)
example.scatter('x', 'y')
resize_window()

In [ ]:
# Put some guidelines on our plot
example = r_table(0.5)
example.scatter('x', 'y')
resize_window()
# Draw a vertical black line at x = 1.5
draw_vertical_line(1.5)
# Draw a red line of slope 1 through (0, 0)
draw_line(slope=1, intercept=0, color='red')

In [ ]:
# Add nearest-neighbor predictions to the example
example = example.with_column('Predicted y', example.apply(nn_prediction_example, 'x'))

# The predictions will now plot in gold
example.scatter('x')
draw_line(slope=1, color='red')
draw_vertical_line(1.5)
resize_window()

# Can you "read" the slope of the gold line from the grid?

In [ ]:
example.scatter('x')
draw_line(slope=1, intercept=0, color='red')

# When r = 0.5, the fit line has slope 0.5
draw_line(slope=0.5, intercept=0)
resize_window()

###  r = 0.7

In [ ]:
example = r_table(0.7)
example = example.with_column('Predicted y', example.apply(nn_prediction_example, 'x'))
example.scatter('x')
draw_line(slope=1, intercept=0, color='red')
# When r = 0.7, the fit line has slope 0.7
draw_line(slope=0.7, intercept=0, color='dodgerblue')
resize_window()

The blue line is called the **regression line** and it demonstrates the "regression effect": In general, individuals who are some distance away from average on one variable are expected to be **not quite as far away from average** on the other. 

In [ ]:
# Back to slides...





## D. Linear regression: defining the line

In [ ]:
# Copy-pasted from above
def standard_units(x):
    """ Converts an array x to standard units """
    return (x - np.mean(x)) / np.std(x)

def correlation(t, x, y):
    """ Computes correlation: t is a table, and x and y are column names """
    x_su = standard_units(t.column(x))
    y_su = standard_units(t.column(y))
    return np.mean(x_su * y_su)


In [ ]:
# By definition, the slope of the regression line ("fit line") in the original units 
# is:  slope = r * y_sd / x_sd
def slope(t, x, y):
    """ Computes the slope of the regression line, like correlation above """
    r = correlation(t, x, y)
    y_sd = np.std(t.column(y))
    x_sd = np.std(t.column(x))
    return r * y_sd / x_sd


In [ ]:
# Test on the example for r = 0.5
# Since the example is in standard units, the slope is essentially equal to r
example = r_table(0.5)
slope(example, 'x', 'y')

In [ ]:
# The regression line passes through the point (x_mean, y_mean) with slope = m,
# so y - y_mean = m * (x - x_mean)
# At the y-intercept, x = 0 and y = y_mean - m * x_mean
def intercept(t, x, y):
    """ Computes the intercept of the regression line """
    x_mean = np.mean(t.column(x))
    y_mean = np.mean(t.column(y))
    
    m = slope(t, x, y)
    return y_mean - m * x_mean

In [ ]:
# Test on the example for r = 0.5
# Since (x_mean, y_mean) = (0, 0) the y-intercept should be 0
intercept(example, 'x', 'y')

## Heights Data and Regression Line
Let's use some real data. Remember, before fitting a regression line or even calculating correlation, it's important to look at the scatter plot (check for a linear association).

In [ ]:
# Note: Child heights are the **adult** heights of children in a family
families = Table.read_table('family_heights.csv')
parent_avgs = (families.column('father') + families.column('mother'))/2
heights = Table().with_columns(
    'Parent Average', parent_avgs,
    'Child', families.column('child'),
)
heights.show(5)

In [ ]:
heights.scatter('Parent Average')
r = correlation(heights, 'Parent Average', 'Child')
print(f'The correlation is {r}')

In [ ]:
# Our nearest-neighbor prediction from earlier...
def nn_prediction_height(p_avg):
    """Predict the height of a child whose parents have a parent average height of p_avg.
    
    The prediction is the average height of the children whose parent average height is
    in the range p_avg plus or minus 0.5.
    """
    
    close_points = heights.where('Parent Average', are.between(p_avg-0.5, p_avg + 0.5))
    return np.average(close_points.column('Child')) 

In [ ]:
# Add the prediction in a column
heights_with_predictions = heights.with_column(
    'Nearest neighbor prediction', 
    heights.apply(nn_prediction_height, 'Parent Average'))
heights_with_predictions.show(5)

In [ ]:
# Visualize the data and the predictions
heights_with_predictions.scatter('Parent Average')

In [ ]:
# Use our `slope` and `intercept` functions with the heights data
predicted_heights_slope = slope(heights, 'Parent Average', 'Child')
predicted_heights_intercept = intercept(heights, 'Parent Average', 'Child')

# The regression line has these parameters, slope and intercept:
[predicted_heights_slope, predicted_heights_intercept]

In [ ]:
# Add a 'Regression Prediction' column
m = predicted_heights_slope
b = predicted_heights_intercept
x = heights.column('Parent Average')

# y = m * x + b
heights_with_predictions = heights_with_predictions.with_column(
    'Regression Prediction', 
    m * x + b
)
heights_with_predictions

In [ ]:
heights_with_predictions.scatter('Parent Average')